In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

# Saved data load and use

In [16]:
# below variables are established to be used throughout the notebook without being subjected to any change
BATCH_SIZE     = 128
HEIGHT = WIDTH = 48
CHANNELS       =  3

EMOTIONS       = ['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']

In [4]:
import os
import re

import numpy as np
import pandas as pd

import cv2
import PIL.Image as Image

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from imblearn.over_sampling import SMOTE

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from torchvision import transforms

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as image

from pprint import pprint

%matplotlib inline

In [37]:
# To load the tensor back
X_smote = np.load('X_smote.npy')
y_smote = np.load('y_smote.npy')

dropping 1 class

In [41]:
X_smote.shape

(12636, 6912)

In [39]:
# Find indices where y is not equal to 6
indices_to_keep = y_smote != 6

# Use boolean indexing to filter x and y
X_smote = X_smote[indices_to_keep]
y_smote = y_smote[indices_to_keep]

# Print the filtered arrays
print("x_filtered:", X_smote)
print("y_filtered:", y_smote)


x_filtered: [[ 46.  46.  46. ... 189. 189. 189.]
 [ 55.  55.  55. ... 187. 187. 187.]
 [123. 123. 123. ...   3.   3.   3.]
 ...
 [ 19.  19.  19. ...  50.  50.  50.]
 [166. 166. 166. ...  31.  31.  31.]
 [126. 126. 126. ...  98.  98.  98.]]
y_filtered: [0 0 0 ... 5 5 5]


In [42]:
y.shape

torch.Size([12636])

In [43]:
set(y_smote)

{0, 1, 2, 3, 4, 5}

In [44]:
from sklearn import preprocessing
# initializing label encoder
label_encoder = preprocessing.LabelEncoder()

# convert X and y to Tensors
x = torch.Tensor(X_smote) # features
y_smote = label_encoder.fit_transform(y_smote) # targets
y = torch.as_tensor(y_smote)

In [45]:
type(x)

torch.Tensor

In [46]:
x.shape

torch.Size([12636, 6912])

In [47]:
# print initial shape of the datasets
print(f"Initial shape of the X datasets along with y:\n\
    {x.shape}, {y.shape}\n")

# reshape to remove the requirements of SMOTE that do not suit standard model training
x = x.reshape((x.shape[0], 3, HEIGHT, WIDTH))

# print latest shape of the datasets
print(f"Latest shape of the X datasets along with y:\n\
    {x.shape}, {y.shape}\n")

Initial shape of the X datasets along with y:
    torch.Size([12636, 6912]), torch.Size([12636])

Latest shape of the X datasets along with y:
    torch.Size([12636, 3, 48, 48]), torch.Size([12636])



In [48]:
type(y)

torch.Tensor

Firstly, define the tasks according to class distribution:

In [49]:
tasks_id = [0 if i<2 else 1 if i<4 else 2 if i<6 else 3 for i in y]


In [50]:
set(tasks_id)

{0, 1, 2}

Define a transformation function to reshape your images, normalized and convert to tensor:

In [51]:
# add data transformations: (1) convert to tensor format, and (2) normalize the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=180),
])

Now, create an Avalanche Tensor Dataset and a Scenario:

In [52]:
# from avalanche.benchmarks.scenarios import NCScenario
# #from avalanche.models import ResNet50
# from avalanche.training.plugins import EvaluationPlugin
# from avalanche.benchmarks.utils import make_tensor_classification_dataset
# #from avalanche.benchmarks.utils.data_loader import ContinuousScenarioDataLoader
# from avalanche.training.supervised import Naive
# from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, forgetting_metrics, disk_usage_metrics, cpu_usage_metrics, timing_metrics
# from torch.optim import SGD
# from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
#     loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
#     disk_usage_metrics, gpu_usage_metrics
# from avalanche.models import SimpleMLP
# from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
# from avalanche.training.plugins import EvaluationPlugin
# from avalanche.training import Naive, Replay


In [53]:
X_train, X_trial, y_train, y_trial = train_test_split(x, y, test_size=0.20, random_state=42)

In [55]:
from avalanche.benchmarks.utils import make_tensor_classification_dataset
trainds = make_tensor_classification_dataset(X_train, y_train)
testds = make_tensor_classification_dataset(X_trial, y_trial)

d:\anaconda\envs\TORCHGPU\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
d:\anaconda\envs\TORCHGPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
trainds

_FlatDataWithTransform (len=10108,subset=False,cat=False,cf=True)
	_TensorClassificationDataset (len=10108)

In [57]:
from avalanche.benchmarks import nc_scenario
from avalanche.training.plugins import EvaluationPlugin
from avalanche.benchmarks.utils import make_tensor_classification_dataset
from avalanche.training.supervised import Naive
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, forgetting_metrics, disk_usage_metrics, cpu_usage_metrics, timing_metrics
from torch.optim import SGD

In [58]:
# split the dataset into tasks
nb_tasks = 3

cl_scenario = nc_scenario(
    train_dataset=trainds,
    test_dataset=testds,
    n_experiences=3,
    task_labels=True,
    seed=1234
)

In [59]:
from torchvision.models import resnet50
from torch.nn import Linear

model = resnet50(pretrained=True)
n_features = model.fc.in_features
model.fc = Linear(n_features, 7)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

d:\anaconda\envs\TORCHGPU\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda\envs\TORCHGPU\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [60]:
# chossing the Naive strategy
cl_strategy = Naive(
    model,
    optimizer,
    torch.nn.CrossEntropyLoss(),
    train_mb_size=500,
    train_epochs=1,
    eval_mb_size=100,
    device=device
)

In [61]:
# Preparing a training plugin
evaluation = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    cpu_usage_metrics(experience=True),
    timing_metrics(),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
)

# here we add a plugin to the strategy
cl_strategy.evaluator = evaluation


d:\anaconda\envs\TORCHGPU\lib\site-packages\avalanche\training\plugins\evaluation.py:94: UserWarning: No loggers specified, metrics will not be logged
  warnings.warn("No loggers specified, metrics will not be logged")


In [62]:
results = []
for experience in cl_scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(cl_scenario.test_stream, num_workers=4))


Start of experience:  0
Current Classes:  [2, 3]
-- >> Start of training phase << --
100%|██████████| 7/7 [00:35<00:00,  5.04s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.5226
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3396
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 9/9 [00:18<00:00,  2.10s/it]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8414
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4814
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 9/9 [00:13<00:00,  1.52s/it]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 4.3603
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 2) from test 